### Importing datasets

In [75]:
from pyspark.sql import SQLContext,Row
from pyspark.sql import functions as F

dataDir = "/home/rsk/Documents/Spark"

In [76]:
userData = sc.textFile(dataDir+"/ml-100k/u.user").map(lambda x : x.split("|"))

In [77]:
movieData = sc.textFile(dataDir+"/ml-100k/u.item").map(lambda x : x.split("|"))

In [78]:
ratingData = sc.textFile(dataDir+"/ml-100k/u.data").map(lambda x : x.split("\t"))

In [79]:
#TO-DO : Convert RDD to Spark DataFrame

ratingDataDF = ratingData.map(lambda x : Row(userID = int(x[0]),
                        movieID = int(x[1]),
                        rating=float(x[2]),
                        timestamp = int(x[3])))
ratingDataDF = sqlContext.createDataFrame(ratingDataDF)

In [80]:
userDataDF = userData.map(lambda x : Row(userID=int(x[0]),
                                        age = int(x[1]),
                                        gender = x[2],
                                        occupation = x[3],
                                        zipcode = x[4]))
userDataDF = sqlContext.createDataFrame(userDataDF)

In [81]:
movieDataDF = movieData.map(lambda x : Row(movieID = int(x[0]),
                                            movieTitle = x[1],
                                            releaseDate = x[2],
                                            videoReleaseDate = x[3],
                                            IMDBurl = x[4],
                                            unknown= int(x[5]),
                                            action = int(x[6]),
                                            adventure = int(x[7]),
                                            animation = int(x[8]),
                                            childrens = int(x[9]),
                                            comedy = int(x[10]),
                                             crime = int(x[11]),
                                             documentary = int(x[12]),
                                             drama = int(x[13]),
                                             fantasy = int(x[14]),
                                             filmNoir = int(x[15]),
                                             horror = int(x[16]),
                                             musical = int(x[17]),
                                             mystery = int(x[18]),
                                             romance = int(x[19]),
                                             sciFi = int(x[20]),
                                             thriller = int(x[21]),
                                             war = int(x[22]),
                                             western = int(x[23])))
movieDataDF = sqlContext.createDataFrame(movieDataDF)
                                             

In [84]:
ratingDataDF.show(5)

+-------+------+---------+------+
|movieID|rating|timestamp|userID|
+-------+------+---------+------+
|    242|   3.0|881250949|   196|
|    302|   3.0|891717742|   186|
|    377|   1.0|878887116|    22|
|     51|   2.0|880606923|   244|
|    346|   1.0|886397596|   166|
+-------+------+---------+------+
only showing top 5 rows



In [ ]:
#Appending Index column to ratingDataDF

indexDF

In [86]:
#Taking smaller subsets of original dataset

u = sqlContext.createDataFrame(userDataDF.rdd.zipWithIndex().filter(lambda x : x[1]<15).map(lambda x : x[0]))



In [95]:
userID = u.map(lambda x : x.userID).collect()

r = ratingDataDF[ratingDataDF.userID.isin(userID)][ratingDataDF.columns]

In [97]:
r.count()

2536

### Merging Datasets

In [101]:
u=u.join(ratingDataDF,u.userID==ratingDataDF.userID,'inner')

In [103]:
u.show()

+---+------+----------+------+-------+-------+------+---------+------+
|age|gender|occupation|userID|zipcode|movieID|rating|timestamp|userID|
+---+------+----------+------+-------+-------+------+---------+------+
| 24|     M|technician|     1|  85711|     61|   4.0|878542420|     1|
| 24|     M|technician|     1|  85711|    189|   3.0|888732928|     1|
| 24|     M|technician|     1|  85711|     33|   4.0|878542699|     1|
| 24|     M|technician|     1|  85711|    160|   4.0|875072547|     1|
| 24|     M|technician|     1|  85711|     20|   4.0|887431883|     1|
| 24|     M|technician|     1|  85711|    202|   5.0|875072442|     1|
| 24|     M|technician|     1|  85711|    171|   5.0|889751711|     1|
| 24|     M|technician|     1|  85711|    265|   4.0|878542441|     1|
| 24|     M|technician|     1|  85711|    155|   2.0|878542201|     1|
| 24|     M|technician|     1|  85711|    117|   3.0|874965739|     1|
| 24|     M|technician|     1|  85711|     47|   4.0|875072125|     1|
| 24| 

In [134]:
data = ratingDataDF.join(userDataDF, userDataDF.userID==ratingDataDF.userID,"inner").drop(userDataDF.userID)

#drop is being used as merging on userID creates two copies of userID in the resulting dataFrame

In [148]:
data

DataFrame[movieID: bigint, rating: double, timestamp: bigint, userID: bigint, age: bigint, gender: string, occupation: string, zipcode: string, IMDBurl: string, action: bigint, adventure: bigint, animation: bigint, childrens: bigint, comedy: bigint, crime: bigint, documentary: bigint, drama: bigint, fantasy: bigint, filmNoir: bigint, horror: bigint, movieTitle: string, musical: bigint, mystery: bigint, releaseDate: string, romance: bigint, sciFi: bigint, thriller: bigint, unknown: bigint, videoReleaseDate: string, war: bigint, western: bigint]

In [130]:
print data.count()
data.show()

100000
+-------+------+---------+------+---+------+----------+-------+
|movieID|rating|timestamp|userID|age|gender|occupation|zipcode|
+-------+------+---------+------+---+------+----------+-------+
|    886|   2.0|881547877|    31| 24|     M|    artist|  10003|
|    484|   5.0|881548030|    31| 24|     M|    artist|  10003|
|    682|   2.0|881547834|    31| 24|     M|    artist|  10003|
|    302|   4.0|881547719|    31| 24|     M|    artist|  10003|
|    135|   4.0|881548030|    31| 24|     M|    artist|  10003|
|    705|   5.0|881548110|    31| 24|     M|    artist|  10003|
|    504|   5.0|881548110|    31| 24|     M|    artist|  10003|
|    498|   4.0|881548111|    31| 24|     M|    artist|  10003|
|    493|   5.0|881548110|    31| 24|     M|    artist|  10003|
|    321|   4.0|881547746|    31| 24|     M|    artist|  10003|
|    514|   5.0|881548030|    31| 24|     M|    artist|  10003|
|    124|   4.0|881548110|    31| 24|     M|    artist|  10003|
|     79|   2.0|881548082|    31|

In [135]:
data = data.join(movieDataDF,data.movieID==movieDataDF.movieID,"inner").drop(movieDataDF.movieID)

In [138]:
print data.count()


100000


In [137]:
data.columns

['movieID',
 'rating',
 'timestamp',
 'userID',
 'age',
 'gender',
 'occupation',
 'zipcode',
 'IMDBurl',
 'action',
 'adventure',
 'animation',
 'childrens',
 'comedy',
 'crime',
 'documentary',
 'drama',
 'fantasy',
 'filmNoir',
 'horror',
 'movieTitle',
 'musical',
 'mystery',
 'releaseDate',
 'romance',
 'sciFi',
 'thriller',
 'unknown',
 'videoReleaseDate',
 'war',
 'western']

In [149]:
data.describe("movieID","rating").show()

+-------+-----------------+-----------------+
|summary|          movieID|           rating|
+-------+-----------------+-----------------+
|  count|           100000|           100000|
|   mean|        425.53013|          3.52986|
| stddev|330.7983563255848|1.125673599144316|
|    min|                1|              1.0|
|    max|             1682|              5.0|
+-------+-----------------+-----------------+



In [151]:
data.select(data.columns[0:3]).show()

+-------+------+---------+
|movieID|rating|timestamp|
+-------+------+---------+
|     31|   4.0|891354612|
|     31|   3.0|880610814|
|     31|   4.0|892334803|
|     31|   5.0|884131157|
|     31|   3.0|890687473|
|     31|   3.0|883391249|
|     31|   4.0|878348998|
|     31|   4.0|880603484|
|     31|   4.0|883948571|
|     31|   4.0|878856526|
|     31|   4.0|879572688|
|     31|   4.0|875807058|
|     31|   4.0|879111937|
|     31|   3.0|887523200|
|     31|   4.0|892679259|
|     31|   5.0|882164867|
|     31|   4.0|882397543|
|     31|   3.0|875148108|
|     31|   3.0|876017533|
|     31|   4.0|891299387|
+-------+------+---------+
only showing top 20 rows



In [156]:
data2 = data.sample(False,0.5,234)

In [158]:
data2.select(data2.columns[0:3]).show()

+-------+------+---------+
|movieID|rating|timestamp|
+-------+------+---------+
|     31|   3.0|880610814|
|     31|   5.0|884131157|
|     31|   4.0|880603484|
|     31|   4.0|883948571|
|     31|   4.0|879111937|
|     31|   4.0|892679259|
|     31|   3.0|875148108|
|     31|   4.0|891299387|
|     31|   4.0|889493628|
|     31|   3.0|884294880|
|     31|   3.0|880314500|
|     31|   4.0|878972119|
|     31|   5.0|880078656|
|     31|   4.0|875310311|
|     31|   3.0|875287615|
|     31|   4.0|885849325|
|     31|   2.0|883546333|
|     31|   4.0|887926573|
|     31|   4.0|874795704|
|     31|   3.0|884753292|
+-------+------+---------+
only showing top 20 rows



In [160]:
data2 = data2.sort(data2.timestamp)

In [162]:
data2.select(data2.columns[0:3]).show()

+-------+------+---------+
|movieID|rating|timestamp|
+-------+------+---------+
|    255|   4.0|874724710|
|    286|   4.0|874724727|
|    298|   4.0|874724754|
|    108|   4.0|874724882|
|    288|   3.0|874724905|
|    117|   4.0|874724988|
|    405|   3.0|874725120|
|   1074|   3.0|874725264|
|    357|   5.0|874725485|
|    210|   4.0|874725485|
|    687|   2.0|874728168|
|    284|   3.0|874728338|
|    147|   4.0|874728461|
|    121|   4.0|874728565|
|    717|   3.0|874728598|
|    220|   5.0|874729682|
|    417|   4.0|874729750|
|    731|   5.0|874729750|
|     50|   4.0|874729750|
|    623|   4.0|874729778|
+-------+------+---------+
only showing top 20 rows

